## Initial data Preprocessing

I have first downloaded the entire dataset and sent all the images to a single folder along side creating a .csv file which contains the path of each image along with the name of the flower the image belongs to as well as the label given to ech flower category.

In [ ]:
import numpy as np 
import pandas as pd

from PIL import ImageFile
from tqdm import tqdm
import cv2

import matplotlib.pylab as plt
from matplotlib import cm
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image as keras_image

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import PReLU, LeakyReLU

## IGNORE THE WARNING

### Image loading and storing as array

In [ ]:
def image_to_tensor(img_path):
    img = keras_image.load_img(r"/kaggle/input/flower-recog-cnn/flower" + img_path, target_size=(128, 128))
    x = keras_image.img_to_array(img)
    return np.expand_dims(x, axis=0)

### Image to tensor

In [ ]:
def data_to_tensor(img_paths):
    list_of_tensors = [image_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True 
# Load the data
data = pd.read_csv('/kaggle/input/flow-csv/Untitled spreadsheet - Sheet1.csv')
files = data['file']
targets = data['label'].values
tensors = data_to_tensor(files);

# The bar graph below shows the loading of images as tensors and storing in form of an array

In [ ]:
data.head()

In [ ]:
tensors.shape

In [ ]:
names = ['Daisy','Dandelions','Rose','Sunflower','Tulip']

## Displaying the image

In [ ]:
def display_images(img_path, ax):
    img = cv2.imread(r"/kaggle/input/flower-recog-cnn/flower" + img_path)
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
fig = plt.figure(figsize=(10,10)) # 10 x 10 inches
for i in range(4): # Only 4 images
    ax = fig.add_subplot(2, 4, i + 1, xticks=[], yticks=[]) # 2 rows and 4 columns and i+1 define sthe position ticks are markings along axes
    ax.set_title(names[targets[i+10]], color='r') # Title to the plot
    display_images(files[i+10], ax)

## Normalize the tensors
As pixel values can having varying intensity, so in order to bring each pixel value within the range of 0-1 we divide each pixel by 255(on a grayscale all intensities are between 0-255)

In [ ]:
tensors = tensors.astype('float32')/255

## One-hot encoding

In [ ]:
targets = to_categorical(targets, 5) 

## Split the data
80% - training set | 10% - testing set | 10% - validation set

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(tensors, targets, 
                                                    test_size = 0.2, 
                                                    random_state = 1)
# Splitting the testing set to validationa and testing set into half
n = int(len(x_test)/2)
x_valid, y_valid = x_test[:n], y_test[:n]
x_test, y_test = x_test[n:], y_test[n:]

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape, y_test.shape

In [ ]:
 x_valid.shape, y_valid.shape

In [ ]:
# Read and display a tensor
print('Label: ', names[np.argmax(y_train[7])])
plt.figure(figsize=(3,3))
plt.imshow((x_train[7]))

## CNN Model

In [ ]:
def model():
    model = Sequential() # Initialise the empty sequential model 

    model.add(Conv2D(128, (3, 3), input_shape=x_train.shape[1:]))
    model.add(LeakyReLU(alpha=0.02))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3)))
    model.add(LeakyReLU(alpha=0.02))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # FINAL LAYER FOR CLASSIFICATION

    model.add(GlobalMaxPooling2D())
    
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Dropout(0.5)) 

    model.add(Dense(5)) # Output layer
    model.add(Activation('softmax'))
    
    # TODO: Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = model()

## Callbacks

In [ ]:
# To save the best model
checkpointer = ModelCheckpoint(filepath='weights.best.model.hdf5',verbose=2, save_best_only=True)

# Earlystopping
early_stop = EarlyStopping(monitor='val_loss', patience=5)

# To reduce learning rate dynamically
lr_reduction = ReduceLROnPlateau(monitor='val_loss',patience=5, verbose=2, factor=0.2)    


In [ ]:
model.summary()

## Training the model

In [ ]:
history = model.fit(x_train, y_train, 
                    epochs=75, batch_size=32, verbose=2,
                    validation_data=(x_valid, y_valid),
                    callbacks=[checkpointer, lr_reduction ,early_stop])

In [ ]:
# Load the model with the best validation accuracy
model.load_weights('weights.best.model.hdf5')

## Evaluation

In [ ]:
# Calculate classification accuracy on the testing set
score = model.evaluate(x_test, y_test)
score

In [ ]:
# Calculate classification accuracy on the validation set
score = model.evaluate(x_valid, y_valid)
score

In [ ]:
# Calculate classification accuracy on the training set
score = model.evaluate(x_train, y_train)
score

## Plots

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
model.save('model.h5')

In [ ]:
model1 = load_model('model.h5')

In [ ]:
# Model predictions for the testing dataset
y_test_predict = model1.predict(x_test)

In [ ]:
y_test_predict = np.argmax(y_test_predict,axis=1)

In [ ]:
# Display true labels and predictions
fig = plt.figure(figsize=(18, 18))
for i, idx in enumerate(np.random.choice(x_test.shape[0], size=16, replace=False)):
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    pred_idx = y_test_predict[idx]
    true_idx = np.argmax(y_test[idx])
    ax.set_title("{} ({})".format(names[pred_idx], names[true_idx]),
                 color=("#4876ff" if pred_idx == true_idx else "darkred"))